In [613]:
from yahoofinancials import YahooFinancials
import numpy as np

In [665]:
class cookFinancials(YahooFinancials):
    ticker = ''
    bshData = []
    def __init__(self, ticker):
        if isinstance(ticker, str):
            self.ticker = ticker.upper()
        else:
            self.ticker = [t.upper() for t in ticker]
        self._cache = {}
        
    def get_balanceSheetHistory(self):
        self.bshData = self.get_financial_stmts('annual', 'balance')['balanceSheetHistory']
        return self.bshData
    
    def get_BV(self, numofYears=20):
        bv = []
        if not(self.bshData):
            self.get_balanceSheetHistory()
        for i in range(min(np.size(self.bshData[self.ticker]), numofYears)):
            date_key = list(self.bshData[self.ticker][i].keys())[0]
            bv.append(self.bshData[self.ticker][i][date_key]['totalStockholderEquity'])
        return bv   
    #use mean of each year    
    def get_BV_GR_median(self, bv):
        gr = []
        for v in range(np.size(bv)-1):
            gr.append((bv[v]-bv[v+1])/abs(bv[v+1]))
        #print(gr)
        return np.size(bv)-1, np.median(gr)
    
    def get_BV_GR_max(self, bv):
        gr = []
        for v in range(np.size(bv)-1):
            gr.append((bv[v]-bv[v+1])/abs(bv[v+1]))
        #print(gr)
        return np.size(bv)-1, np.max(gr)
    
    def get_BV_GR_mean(self, bv):
        gr = []
        for v in range(np.size(bv)-1):
            gr.append((bv[v]-bv[v+1])/abs(bv[v+1]))
        #print(gr)
        return np.size(bv)-1, np.mean(gr)
    
    def get_suggest_price(self, cEPS, growth, years, rRate, PE, safty):
        if not(cEPS) or not(growth) or not(PE):
            return 'NA'
        fEPS = cEPS*(1+growth)**years
        fPrice = fEPS*PE;
        stickerPrice = fPrice/(1+rRate)**years
        return stickerPrice, stickerPrice*safty
    
    def get_PE(self):
        #print(self._stock_summary_data('trailingPE'))
        #print(self._stock_summary_data('forwardPE'))
        if not(self._stock_summary_data('trailingPE')):
            return self._stock_summary_data('forwardPE')
        return (self._stock_summary_data('trailingPE')+self._stock_summary_data('forwardPE'))/2
    
    def get_decision(self,suggestPrice, stockprice):
        print('suggested price:', suggestPrice)
        print('stock price:', stockprice)
        if isinstance(suggestPrice, str):
            return 'no recommendation due to error'
        elif suggestPrice>stockprice:
            return 'strong buy' 
        else:
            return 'do not buy'
            
            
        

In [666]:
class batch_process:
    def __init__(self, tickers):
        self.tickers = tickers
    def batch_run(self):
        for i in range(np.size(self.tickers)):
            print(self.tickers[i])
            x = cookFinancials(self.tickers[i])
            bv = x.get_BV(20)
            bv.insert(0, x.get_book_value())
            #print(bv)
            bvgr = x.get_BV_GR_median(bv)
            print(bvgr)
            growth = bvgr[1]
            cEPS = x.get_earnings_per_share()
            years = 3;
            rRate = 0.2;
            safty = 0.5
            PE = x.get_PE()
            price=x.get_suggest_price(cEPS, growth, years, rRate, PE, safty)
            #print(price)
            print(x.get_decision(price[1],x.get_current_price()))
            print('=====================================')

In [615]:
x = cookFinancials('TSLA')

In [616]:
bv = x.get_BV(20)

In [617]:
bv.insert(0, x.get_book_value())
print(bv)

[16031000000, 6618000000, 4923000000, 4237242000, 4752911000]


In [618]:
bvgr = x.get_BV_GR_median(bv)
print(bvgr)

[1.4223330311272289, 0.34430225472273, 0.16184065012099852, -0.10849540418493003]
(4, 0.25307145242186424)


In [619]:
growth = bvgr[1]
cEPS = x.get_earnings_per_share()
years = 3;
rRate = 0.2;
safty = 0.5
PE = x.get_PE()

1087.61
145.85129


In [620]:
print(x.get_earnings_per_share())


0.5229999724165832


In [621]:
x.get_pe_ratio()
x.get_PE()

1087.61
145.85129


616.730645

In [622]:
print(cEPS)
print(growth)
print(years)
print(rRate)
print(PE)
print(safty)

0.5229999724165832
0.25307145242186424
3
0.2
616.730645
0.5


In [623]:
price=x.get_suggest_price(cEPS, growth, years, rRate, PE, safty)
print(price)

(367.26620257306394, 183.63310128653197)


In [624]:
x.get_decision(price[1],x.get_current_price())

183.63310128653197
568.82


'do not buy'

In [628]:
from get_all_tickers import get_tickers as gt
from get_all_tickers.get_tickers import Region

In [633]:
tickers = gt.get_tickers(AMEX=False)



['DDD', 'MMM', 'WBAI', 'EGHT', 'AHC']

In [ ]:
tickers[:5]

In [642]:
filtered_by_sector = gt.get_tickers_filtered(mktcap_min=10e3, sectors=gt.SectorConstants.HEALTH_CARE)

In [658]:
print(filtered_by_sector)

['MMM', 'ABT', 'ABBV', 'ALC', 'ABC', 'ANTM', 'BAX', 'BDX', 'BSX', 'BMY', 'CAH', 'CTLT', 'CNC', 'CRL', 'CI', 'CVS', 'DVA', 'RDY', 'EW', 'ELAN', 'LLY', 'FMS', 'GSK', 'HCA', 'HUM', 'IQV', 'JNJ', 'LH', 'MCK', 'MDT', 'MRK', 'MOH', 'NVS', 'NVO', 'OSH', 'PFE', 'QGEN', 'DGX', 'RMD', 'SNN', 'STE', 'SYK', 'TAK', 'TDOC', 'TFX', 'TEVA', 'COO', 'UNH', 'UHS', 'VAR', 'ZBH', 'ZTS', 'ABMD', 'ALXN', 'ALGN', 'ALNY', 'AMGN', 'ARGX', 'AZN', 'BGNE', 'BIIB', 'BMRN', 'BNTX', 'TECH', 'CVAC', 'XRAY', 'DXCM', 'EXAS', 'GMAB', 'GILD', 'GRFS', 'GH', 'HOLX', 'HZNP', 'ICLR', 'IDXX', 'INCY', 'PODD', 'ISRG', 'MASI', 'MRTX', 'MRNA', 'NVCR', 'PPD', 'QGEN', 'REGN', 'RGEN', 'RPRX', 'SNY', 'SRPT', 'SGEN', 'VRTX', 'VTRS', 'WBA']


In [660]:
filtered_by_sector[1]

'ABT'

In [667]:
y = batch_process(filtered_by_sector)

In [ ]:
y.batch_run()

MMM
(4, 0.07504770457753791)
suggested price: 58.76441691021123
stock price: 171.85
do not buy
ABT
(4, 0.014031478010469122)
suggested price: 23.326206379613332
stock price: 108.15
do not buy
ABBV
(4, 0.06594028203741498)
suggested price: 25.2967640192931
stock price: 104.99
do not buy
ALC
(4, -0.02522663734711066)
suggested price: A
stock price: 63.19
no recommendation due to error
ABC
(4, -0.009190288950172256)
suggested price: A
stock price: 103.12
no recommendation due to error
ANTM
(4, 0.07300785725853089)
suggested price: 100.57162027755349
stock price: 318.45
do not buy
BAX
(4, 0.04587567176796038)
suggested price: 18.96272569510114
stock price: 76.35
do not buy
BDX
(4, 0.06573747725057826)
suggested price: 50.975124588655596
stock price: 241.87
do not buy
BSX
(4, 0.18300308418255884)
suggested price: 21.0940622310757
stock price: 34.15
do not buy
BMY
(4, 0.08356747660831704)
suggested price: A
stock price: 62.18
no recommendation due to error
CAH
(4, -0.15758008202627494)
sugge